In [1]:
import pandas as pd
import xgboost as xgb
import numpy as np
from xgboost import plot_tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import Imputer
from sklearn.pipeline import FeatureUnion
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import FeatureUnion
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import average_precision_score, precision_recall_curve
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [2]:
train_identity = pd.read_csv('train_identity.csv.zip')
train_transaction = pd.read_csv('train_transaction.csv.zip')
print(train_identity.shape)
print(train_transaction.shape)
data = pd.merge(train_transaction, train_identity, on='TransactionID', how='left')
data = data.drop('TransactionID', axis=1)
print(data.shape)

(144233, 41)
(590540, 394)
(590540, 433)


In [3]:
X, y = data.iloc[:,data.columns!='isFraud'],data.loc[:,data.columns=='isFraud']
print(X.shape)
print(y.shape)

(590540, 432)
(590540, 1)


In [4]:
# Check number of nulls in each feature column
nulls_per_column = X.isnull().sum()
print(nulls_per_column)

TransactionDT          0
TransactionAmt         0
ProductCD              0
card1                  0
card2               8933
                   ...  
id_36             449555
id_37             449555
id_38             449555
DeviceType        449730
DeviceInfo        471874
Length: 432, dtype: int64


In [5]:
# Create a boolean mask for categorical columns
categorical_feature_mask = X.dtypes == object

In [6]:
# Get list of categorical column names
categorical_columns = X.columns[categorical_feature_mask].tolist()

In [7]:
# Get list of non-categorical column names
non_categorical_columns = X.columns[~categorical_feature_mask].tolist()

In [8]:
# Apply numeric imputer
numeric_imputation_mapper = DataFrameMapper(
                                            [([numeric_feature], SimpleImputer(strategy="median")) for numeric_feature in non_categorical_columns],
                                            input_df=True,
                                            df_out=True
                                           )

In [9]:
# Apply categorical imputer
categorical_imputation_mapper = DataFrameMapper(
                                                [(category_feature, CategoricalImputer()) for category_feature in categorical_columns],
                                                input_df=True,
                                                df_out=True
                                               )

In [10]:
# Combine the numeric and categorical transformations
numeric_categorical_union = FeatureUnion([
                                          ("num_mapper", numeric_imputation_mapper),
                                           ("cat_mapper", categorical_imputation_mapper)
                                         ])

In [11]:
# Custom transformer to convert Pandas DataFrame into Dict (needed for DictVectorizer)
class Dictifier(BaseEstimator, TransformerMixin): 
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = pd.DataFrame(X)
        return X.to_dict('records')



# Create the DictVectorizer object: dv
dv = DictVectorizer(sort=False)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic',n_estimators=10,seed=123)



# Create full pipeline
pipeline = Pipeline([
                     ("featureunion", numeric_categorical_union),
                       ("dictifier", Dictifier()),
                       ("vectorizer", dv),
                        ("clf", xg_cl)
                    ])



# Perform cross-validation
cross_val_scores = cross_val_score(pipeline, X, y, scoring="average_precision", cv=3)

# Print avg. AUC
print("3-fold average precision: ", np.mean(cross_val_scores))

/home/meslamib/env/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/meslamib/env/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/meslamib/env/lib/python3.7/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/meslamib/env/lib/python3.7/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the s

3-fold average precision:  0.33762588486798933


In [ ]:
# Fit the classifier to the training set
pipeline.fit(X, y)

# Visualizing feature importances

xgb.plot_importance(xg_cl)
plt.show()